In [11]:
using ArchGDAL
using FreqTables
using LinearAlgebra
using DataFrames
using Dates
using Pipe
using Plots
using Rasters
using Serialization
using SparseArrays

include("util.jl")
include("geo_ana.jl")
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [12]:
path_spatial = "../dt_tmp/spatial_params_agg230.ser"

"../dt_tmp/spatial_params_agg230.ser"

## International airport location, index 

In [14]:
sp_pars = deserialize(path_spatial)
df = sp_pars.df
# Tambo International
lat = -26.12825796201514
lon = 28.242074092511
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# Cape Town 
lat =  -33.970502228847884
lon = 18.600228711334545
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")

# King Shaka 
lat =  -29.608764960536764
lon = 31.115368797913593
dist = harversine_dist.(lat, lon, df[:, :lat], df[:, :lon]) 
println("argmin(dist): $(argmin(dist)), dist: $(dist[argmin(dist)])")



argmin(dist): 11, dist: 9.631110668981826
argmin(dist): 7, dist: 5.508420257007044
argmin(dist): 62, dist: 8.80101225388816


# Transmission model 

In [15]:
n_sim = 5000

5000

In [16]:
R0 = 14.0
α = 0.05
pc = 0.25
pattern = "population_size"

"population_size"

In [8]:
imp_ws_moz = CSV.read("../data/imp_ws_moz.csv", DataFrame, header=false)[:,1]
imp_ws_airport = CSV.read("../data/imp_ws_airport.csv", DataFrame, header=false)[:,1]
nothing

In [9]:
include("model_meta_pop.jl")
include("main_model_meta_pop.jl")

validate_meta_population_model

In [10]:
function post_processing(path_trans, path_spatial, pc)
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    #validate_meta_population_model(path_trans, path_spatial)
    remove_all_leaving_one(path_trans)
    return path_save
end

function run_all_patterns(;
        R0=1.0, α=0.05, pc=1.0, 
        imp_ws_moz=[1.0], imp_ws_airport=[1.0],
        n_sim=100,
        path_spatial=""
    )
    summary_info = []
    for pattern in ["population_size", "airport", "mozambique"]
        imp_ws = pattern=="airport" ? imp_ws_airport : imp_ws_moz
        path_trans = run_transmission_model(;
            R0=R0, α=α, pc=pc, imp_ws=imp_ws,
            n_sim=n_sim,
            path_spatial_params=path_spatial, 
            pattern=pattern
        )
        path_save = post_processing(path_trans, path_spatial, pc)
        rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
        push!(summary_info, rec)
    end
    println(summary_info)
end

run_all_patterns (generic function with 1 method)

# Main results 

In [ ]:
println("Main results==========================")
run_all_patterns(
    R0=R0, α=α, pc=pc, imp_ws_moz=imp_ws_moz, imp_ws_airport=imp_ws_airport, 
    n_sim=n_sim,
    path_spatial=path_spatial,
    )

## Sensitivity analysis 

In [ ]:
println("Sensitivity analysis for R0 ==========================")
summary_info = []
pattern = "population_size"
for R0_sens in [10, 12, 16]
    path_trans = run_transmission_model(;
        R0=R0_sens, α=α, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0_sens), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

In [ ]:
println("Sensitivity analysis for α ==========================")
summary_info = []
for α_sens in [0.005, 0.010, 0.100, 0.500]
    path_trans = run_transmission_model(;
        R0=R0, α=α_sens, pc=pc, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc)
    rec = "R0=$(R0), α=$(α_sens), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

In [11]:
println("Sensitivity analysis for pc ==========================")
summary_info = []
for pc_sens in [0.01, 0.05, 0.25, 0.50]
    path_trans = run_transmission_model(;
        R0=R0, α=α, pc=pc_sens, imp_ws=[1.0],
        n_sim=n_sim,
        path_spatial_params=path_spatial, 
        pattern=pattern,
    )
    path_save = post_processing(path_trans, path_spatial, pc_sens)
    rec = "R0=$(R0), α=$(α), pc=$(pc_sens), n_sim=$(n_sim), pattern=$(pattern), path: $(path_save)"
    push!(summary_info, rec)
end
println(summary_info)

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.01
  imp_ws: Array{Float64}((1,)) [1.0]
Average va

Progress: 100%|█████████████████████████████████████████| Time: 1:27:46


5266.548859 seconds (58.96 M allocations: 17.135 TiB, 0.83% gc time)
Cum index:1517
Actual ES coverage: 100.0
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:17:39


../dt_tmp_res/20231126_042455.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.05
  imp_ws: Arr

...

../dt_tmp_res/20231126_074457.ser
# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.5
  imp_ws: Arra

Progress: 100%|█████████████████████████████████████████| Time: 1:48:55


6535.231728 seconds (58.19 M allocations: 16.835 TiB, 11.91% gc time)
Cum index:9
Actual ES coverage: 16.731243
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:27:21


../dt_tmp_res/20231126_100150.ser
Any["R0=14.0, α=0.05, pc=0.01, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231126_042455.ser", "R0=14.0, α=0.05, pc=0.05, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231126_060455.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231126_074457.ser", "R0=14.0, α=0.05, pc=0.5, n_sim=5000, pattern=population_size, path: ../dt_tmp_res/20231126_100150.ser"]


## Mozambique risk ordered ES. 

In [17]:
imp_ws_moz_sorted = CSV.read("../data/imp_ws_moz_sorted.csv", DataFrame, header=false)[:,1]
imp_ws_airport_sorted = CSV.read("../data/imp_ws_airport_sorted.csv", DataFrame, header=false)[:,1]
path_spatial_sorted = "../dt_tmp/spatial_params_agg230_moz_sorted.ser"

"../dt_tmp/spatial_params_agg230_moz_sorted.ser"

In [ ]:
println("Sensitivity analysis for Mozambique risk ordered ES ==========================")
run_all_patterns(
    R0=R0, α=α, pc=pc, 
    imp_ws_moz=imp_ws_moz_sorted, 
    imp_ws_airport=imp_ws_airport_sorted,
    n_sim=n_sim,
    path_spatial=path_spatial_sorted,
    )

# Detection sensitivity, sensitivity analysis

In [10]:
println("Sensitivity analysis for pop90 ==========================")
summary_info = []
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=[1.0],
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern=pattern
)
for pop90 in [1, 30, 100, 300]
    g = - log(1-0.9)/pop90
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc, g=g)
    
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), pop90=$(pop90), path: $(path_save)"
    push!(summary_info, rec)
end
remove_all_leaving_one(path_trans)
print(summary_info)


# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1,)) [1.0]
Average va

Progress: 100%|█████████████████████████████████████████| Time: 1:22:16


4936.390346 seconds (56.37 M allocations: 16.211 TiB, 1.13% gc time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 2.302585092994046
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:15:22


../dt_tmp_res/20231126_015148.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.07675283643313487
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


...

../dt_tmp_res/20231126_020725.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.02302585092994046
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:15:45


../dt_tmp_res/20231126_022310.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.0076752836433134864
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 30


Progress: 100%|█████████████████████████████████████████| Time: 0:15:48


../dt_tmp_res/20231126_023859.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, pop90=1, path: ../dt_tmp_res/20231126_015148.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, pop90=30, path: ../dt_tmp_res/20231126_020725.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, pop90=100, path: ../dt_tmp_res/20231126_022310.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, pop90=300, path: ../dt_tmp_res/20231126_023859.ser"]

# Sampling frequency sensitivity

In [9]:
println("Sensitivity analysis for sampling frequency ==========================")
summary_info = []
path_trans = run_transmission_model(;
    R0=R0, α=α, pc=pc, imp_ws=[1.0],
    n_sim=n_sim,
    path_spatial_params=path_spatial, 
    pattern=pattern
)
for n_freq in [1, 7, 14, 60]
    par_AFP, par_ES = set_par_AFP_ES(path_spatial_params=path_spatial, pc=pc, n_freq=n_freq)
    path_save = sensitivity_all_summary(par_AFP, par_ES; path_trans=path_trans)
    rec = "R0=$(R0), α=$(α), pc=$(pc), n_sim=$(n_sim), pattern=$(pattern), n_freq=$(n_freq), path: $(path_save)"
    push!(summary_info, rec)
end
remove_all_leaving_one(path_trans)
print(summary_info)

# of sites: 1517
SEIRMetaModelParams
  R0: Float64 14.0
  γ1: Float64 0.25
  γ2: Float64 0.06657789613848203
  σ: Float64 0.3291
  P_AFP: Float64 0.005
  N_tot: Array{Int64}((1517,)) [176661, 132438, 110952, 102279, 96377, 93257, 81002, 79789, 74368, 71796  …  103, 102, 102, 102, 102, 102, 102, 101, 101, 101]
  N_unvac: Array{Int64}((1517,)) [9009, 7004, 10801, 6729, 6341, 22933, 4284, 4219, 4893, 3975  …  4, 7, 7, 19, 19, 12, 12, 6, 7, 4]
  I0_init: Int64 1
  I0_ind: Int64 1
  days: Int64 1095
  n_site: Int64 1517
  α: Float64 0.05
  π_mat: Array{Float64}((1517, 1517)) [0.0 0.0008134913063202908 … 6.780709468953783e-7 4.898477234903362e-6; 0.0034150940977343493 0.0 … 4.753101976878339e-5 4.686095869451221e-6; … ; 1.9892426060623706e-6 0.00011196468975607586 … 0.0 3.3899886725864443e-9; 2.770651210640693e-5 4.626887381318493e-7 … 3.88258866975442e-10 0.0]
  μ: Float64 0.000547945205479452
  β: Float64 0.9320905459387484
  pc: Float64 0.25
  imp_ws: Array{Float64}((1,)) [1.0]
Average va

Progress: 100%|█████████████████████████████████████████| Time: 1:43:50


6230.282390 seconds (57.74 M allocations: 16.211 TiB, 2.06% gc time, 0.01% compilation time)
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 1


Progress: 100%|█████████████████████████████████████████| Time: 1:20:53


../dt_tmp_res/20231125_231613.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 7


...

../dt_tmp_res/20231125_233927.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 14


Progress: 100%|█████████████████████████████████████████| Time: 0:19:35


../dt_tmp_res/20231125_235903.ser
Cum index:34
Actual ES coverage: 34.42009
AFPSurParams
  P_test: Float64 0.97
  P_sample: Float64 0.53
  P_H: Float64 0.9
ESParams
  g: Float64 0.23025850929940458
  P_test: Float64 0.97
  area: Array{Bool}((1517,)) Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  n_freq: Int64 60


Progress: 100%|█████████████████████████████████████████| Time: 0:14:35


../dt_tmp_res/20231126_001339.ser
Any["R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, n_freq=1, path: ../dt_tmp_res/20231125_231613.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, n_freq=7, path: ../dt_tmp_res/20231125_233927.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, n_freq=14, path: ../dt_tmp_res/20231125_235903.ser", "R0=14.0, α=0.05, pc=0.25, n_sim=5000, pattern=population_size, n_freq=60, path: ../dt_tmp_res/20231126_001339.ser"]